In [2]:
#https://www.cursor.com/
#https://docs.anaconda.com/miniconda/
#conda create -n myenv python=3.10 (eu estou usando python 3.12.5)



O script implementa uma versão simplificada do modelo Transformer, que é uma arquitetura de rede neural usada principalmente para tarefas de processamento de linguagem natural. Aqui estão os principais componentes:
1. Função softmax: Converte um vetor de números em uma distribuição de probabilidade.
Atenção de produto escalar escalado: Calcula a atenção entre consultas (Q), chaves (K) e valores (V).
Atenção Multi-Cabeça: Permite que o modelo se concentre em diferentes partes da entrada simultaneamente.
Rede Feed-Forward: Processa as saídas da camada de atenção.
Camada do Codificador: Combina a atenção multi-cabeça e a rede feed-forward.
Transformer: Combina várias camadas de codificador e adiciona uma camada de embedding.
O fluxo de dados no modelo é o seguinte:
A entrada é convertida em embeddings.
Cada camada do codificador processa os dados:
a. A atenção multi-cabeça calcula a atenção.
b. A rede feed-forward processa o resultado da atenção.
3. A saída final representa a codificação da sequência de entrada.


In [3]:
import numpy as np

# Função softmax para converter valores em probabilidades
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Função de atenção de produto escalar escalado
def scaled_dot_product_attention(Q, K, V):
    # Calcular o fator de escala
    d_k = K.shape[-1]
    # Calcular os scores de atenção
    attention_scores = np.einsum('...ij,...kj->...ik', Q, K) / np.sqrt(d_k)
    # Aplicar softmax para obter as probabilidades de atenção
    attention_probs = softmax(attention_scores)
    # Calcular a saída ponderada
    output = np.einsum('...ij,...jk->...ik', attention_probs, V)
    return output

# Classe de Atenção Multi-Cabeça
class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads
        
        # Inicializar as matrizes de peso
        self.WQ = np.random.randn(d_model, d_model)
        self.WK = np.random.randn(d_model, d_model)
        self.WV = np.random.randn(d_model, d_model)
        self.WO = np.random.randn(d_model, d_model)
    
    # Função para dividir as cabeças de atenção
    def split_heads(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, self.num_heads, self.depth)
        return x.transpose(0, 2, 1, 3)
    
    # Função de propagação para frente
    def forward(self, Q, K, V):
        batch_size = Q.shape[0]
        
        # Aplicar as transformações lineares
        Q = np.matmul(Q, self.WQ)
        K = np.matmul(K, self.WK)
        V = np.matmul(V, self.WV)
        
        # Dividir em múltiplas cabeças
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        # Calcular a atenção
        attention_output = scaled_dot_product_attention(Q, K, V)
        # Reorganizar e concatenar as cabeças
        attention_output = attention_output.transpose(0, 2, 1, 3).reshape(batch_size, -1, self.d_model)
        
        # Aplicar a transformação linear final
        output = np.matmul(attention_output, self.WO)
        return output

# Classe da rede Feed-Forward
class FeedForward:
    def __init__(self, d_model, d_ff):
        self.W1 = np.random.randn(d_model, d_ff)
        self.W2 = np.random.randn(d_ff, d_model)
    
    def forward(self, x):
        # Aplicar duas transformações lineares com ReLU no meio
        return np.matmul(np.maximum(np.matmul(x, self.W1), 0), self.W2)

# Classe da Camada do Codificador
class EncoderLayer:
    def __init__(self, d_model, num_heads, d_ff):
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff)
    
    def forward(self, x):
        # Aplicar atenção multi-cabeça
        attention_output = self.mha.forward(x, x, x)
        # Aplicar rede feed-forward
        ff_output = self.ff.forward(attention_output)
        return ff_output

# Classe principal do Transformer
class Transformer:
    def __init__(self, num_layers, d_model, num_heads, d_ff, vocab_size):
        self.embedding = np.random.randn(vocab_size, d_model)
        # Criar várias camadas de codificador
        self.layers = [EncoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)]
    
    def forward(self, x):
        # Converter índices em embeddings
        x = self.embedding[x]
        # Passar pelos codificadores
        for layer in self.layers:
            x = layer.forward(x)
        return x

# Exemplo de uso
vocab_size = 1000
d_model = 64
num_heads = 4
d_ff = 128
num_layers = 2

# Criar o modelo Transformer
transformer = Transformer(num_layers, d_model, num_heads, d_ff, vocab_size)

# Criar uma entrada de exemplo (lote de sequências de índices de tokens)
input_seq = np.random.randint(0, vocab_size, size=(2, 10))  # 2 sequências, cada uma com 10 tokens

# Processar a entrada através do modelo
output = transformer.forward(input_seq)
print("Forma da entrada:", input_seq.shape)
print("Forma da saída:", output.shape)

Forma da entrada: (2, 10)
Forma da saída: (2, 10, 64)



# Explicação do Transformer Simplificado

## Visão Geral
Este script implementa uma versão simplificada do modelo Transformer, uma arquitetura de rede neural amplamente utilizada em tarefas de processamento de linguagem natural.

## Componentes Principais

1. **Função softmax**
   - Converte um vetor de números em uma distribuição de probabilidade.

2. **Atenção de Produto Escalar Escalado**
   - Calcula a atenção entre consultas (Q), chaves (K) e valores (V).

3. **Atenção Multi-Cabeça**
   - Permite que o modelo se concentre em diferentes partes da entrada simultaneamente.

4. **Rede Feed-Forward**
   - Processa as saídas da camada de atenção.

5. **Camada do Codificador**
   - Combina a atenção multi-cabeça e a rede feed-forward.

6. **Transformer**
   - Combina várias camadas de codificador e adiciona uma camada de embedding.

## Fluxo de Dados

1. A entrada é convertida em embeddings.
2. Cada camada do codificador processa os dados:
   a. A atenção multi-cabeça calcula a atenção.
   b. A rede feed-forward processa o resultado da atenção.
3. A saída final representa a codificação da sequência de entrada.

## Funcionamento

- O modelo recebe uma sequência de índices de tokens como entrada.
- Esses índices são convertidos em vetores de embedding.
- As camadas de codificador processam esses embeddings, aplicando atenção e transformações feed-forward.
- A saída final é uma representação codificada da sequência de entrada.

## Observações

- Este é um modelo simplificado para fins didáticos.
- Não inclui características avançadas como mascaramento, normalização de camadas ou treinamento.
- Serve como uma introdução aos conceitos fundamentais do Transformer.